In [2]:
import pandas as pd
import selenium
import csv
import requests
import traceback
import sys,os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import psycopg2
import numpy as np
import psycopg2.extras as extras
from io import StringIO


STEPS:
1 - Gerar requests dos arquivos a serem baixados (Extract - Depois específicar método que foi utilizado para obter as URLS no portfólio) 
2 - Executar Tratamento (append nas datas ignorandos os headers - Analisar se os headers são constantes)
3 - gerar Análise do CSV tratado (Load)
4 - Montagem do BI com os dados com kpis que são relevantes

In [3]:
#Á análise consta apenas anos de 2010 à 2021
#criando lista de urls a serem baixadas

url_list = []
for i in range(10,22):
    
    a = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-20'+str(i)+'-01.csv'
    url_list.append(a)
    b = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-20'+str(i)+'-02.csv'
    url_list.append(b)
       
    

In [9]:

df_res = pd.DataFrame()
for ur in url_list:

    try:


        with requests.Session() as s:
            print('iniciando processo da url '+str(ur))
            
            download = s.get(ur)

            decoded_content = download.content.decode('utf-8')

            cr = csv.reader(decoded_content.splitlines(), delimiter=';')

            my_list = list(cr)

            df = pd.DataFrame(my_list)
            
            df.columns = df.iloc[0]

            df = df.iloc[1:] 

            print('Csv de tamanho '+str(len(df)))

            df_res = df_res.append(df)

            print('sucesso na url '+str(ur)+'')

    except Exception as e:

        exc_type, exc_obj, exc_tb = sys.exc_info()

        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]

        print(exc_type, fname, exc_tb.tb_lineno)
        print('problema na url'+str(ur))
        pass
        


iniciando processo da url https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2010-01.csv
Csv de tamanho 628516
sucesso na url https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2010-01.csv
iniciando processo da url https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2010-02.csv
Csv de tamanho 645339
sucesso na url https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2010-02.csv
iniciando processo da url https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2011-01.csv
Csv de tamanho 649693
sucesso na url https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2011-01.csv
iniciando processo da url https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2011-02.csv
Csv de tamanho 20468
sucesso na url https://www.gov.br/anp/pt-br/centrais-de-c

In [2]:
df_res.to_csv('res_final.csv')

NameError: name 'df_res' is not defined

In [4]:

len(df_res)

6947169

In [3]:
df_res = pd.read_csv('res_final.csv',delimiter=',')

C:\Users\arald\AppData\Local\Temp\ipykernel_22532\2458343303.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res = pd.read_csv('res_final.csv',delimiter=',')


In [4]:
dfy = df_res.copy()
dfy.drop(['Unnamed: 0'],axis='columns',inplace=True)

Montando tabela no Banco de Dados

In [40]:
connection = psycopg2.connect(user="dbadmin",
                                  password="abc123",
                                  host="localhost",
                                  port="5432",
                                  database="postgresDB")
cursor = connection.cursor()

postgres_create_query = """CREATE TABLE schema_custo."dw_custo_gasolina"(
    Regiao text,
    Estado text,
    Municipio text,
    Revenda text,
    CNPJ_da_revenda text,
    Nome_da_Rua text,
    Numero_da_rua text,
    Complemento text,
    Bairro text,
    Cep text,
    Produto text,
    Data_da_Coleta text,
    Valor_de_Venda text,
    Valor_de_Compra text,
    Unidade_de_Medida text,
    Bandeira text
    ); """
cursor.execute(postgres_create_query)
connection.commit()
print('tabela criada com sucesso')
cursor.close()
connection.close()

"""Especificar que os tipos das colunas foram usados como texto apenas por facilidade específica no upload do dataframe,
    não é uma pratica comum que deve ser seguida pois acaba por se perder em ineficiência a utilização do banco de dados.
"""
########## NOTA :  USAR MÉTODO DE ALTER COLUMN TYPE PARA VERIFICAR RESULTADO SE EU CONSIGO RECUPERAR EFICIÊNCIA DA BASE DE DADOS

tabela criada com sucesso


In [14]:
dfy['Data da Coleta'] = pd.to_datetime(dfy['Data da Coleta'],errors= 'coerce')
dfy['Data da Coleta'] = dfy['Data da Coleta'].dt.strftime('%d/%m/%Y')
dfy['Data da Coleta']

0          01/05/2010
1          01/05/2010
2          01/05/2010
3          01/05/2010
4          01/04/2010
              ...    
4681529    28/06/2021
4681530    28/06/2021
4681531    28/06/2021
4681532    28/06/2021
4681533    28/06/2021
Name: Data da Coleta, Length: 4681534, dtype: object

In [6]:
dft = dfy.copy()


Levando os dados para o banco de 1000 em 1000 linhas

In [41]:



connect = psycopg2.connect(user="dbadmin",
                                  password="abc123",
                                  host="localhost",
                                  port="5432",
                                  database="postgresDB")


def execute_batch(con, df, table, page_size=1000):
    """
    Using psycopg2.extras.execute_batch() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = con.cursor()
    try:
        extras.execute_batch(cursor, query, tuples, page_size)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cursor.close()
        return 1
    print("execute_batch() done")
    cursor.close()

dfy = dfy.rename(columns={
    "\ufeffRegiao": "Regiao",
    "CNPJ da Revenda": "cnpj_da_revenda",
    "Nome da Rua": "nome_da_rua",
    "Numero Rua": "numero_da_rua",
    "Data da Coleta": "data_da_coleta",
    "Valor de Venda": "valor_de_venda",
    "Valor de Compra": "valor_de_compra",
    "Unidade de Medida": "unidade_de_medida"
})
#O python conhece como float, valores com '.' e como em pt-br nossos valores estão embutidos com ',' irei upar ao banco como string e corrigirei o
# problema mais a frente


execute_batch(connect,dfy,'schema_custo.dw_custo_gasolina')


execute_batch() done
